In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests 
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import openpyxl
from urllib.parse import urljoin
from selenium.common.exceptions import TimeoutException
from datetime import datetime

In [9]:
url = "https://www.asiaone.com/tags/mental-health"

In [33]:
def get_page_data(url,keywords):
    
    source = requests.get(url).content
    while(source == None):
        source = requests.get(url).content
    soup = BeautifulSoup(source,"lxml")
    title = soup.find("h1",class_ ="title").text 
    date_str = soup.find("span",class_ ="datetime").text
    content = soup.find("div",class_ = "body").text.replace("\n"," ") if soup.find("div",class_= "body") != None else "None"
    author = soup.find("a",class_= "last-name").text if soup.find("a",class_= "last-name") != None else "None" 
    related_article = False
    keyfound = False

    
    for key in keywords:
        if key in content.lower():
            related_article = True
            
        if key in title.lower():
            related_article = True
            
            
    if "covid" in content.lower():
        keyfound = True
    elif "covid" in title.lower():
        keyfound = True

        
    if related_article:
        return [title,url,author,date_str,content,keyfound]
        

In [12]:
path = "C:\Program Files\chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get(url)

sleep(2)
scroll_pause_time = 1 
screen_height = driver.execute_script("return window.screen.height;")
i = 1
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break
        
        
#clicking the load more button        
while True:
    try:
        load = driver.find_element_by_class_name("ao-btn")
        load.click()
        sleep(2)
    except Exception as e:
        print("complete")
        break
sleep(3)

urls = []
soup = BeautifulSoup(driver.page_source, "lxml")
for parent in soup.find_all(class_="ant-list-item"):
    a_tag = parent.find("a", route="Article")

    try:
        aSource = parent.find("div",class_="ant-card-meta-description").text 
        aSource = aSource.lower()
    except Exception:
        aSource = "none"
    base = "https://www.asiaone.com/"
    link = a_tag.attrs['href']
    
    if(aSource == "asiaone" or aSource == "none"):
        urlC = urljoin(base, link)
        urls.append(urlC)
driver.close()

complete


In [13]:
urls


['https://www.asiaone.com/lifestyle/it-took-months-see-end-tunnel-gymnast-opens-about-how-her-coaches-caused-her-eating',
 'https://www.asiaone.com/singapore/everyone-watching-makes-me-feel-uneasy-students-speak-mental-health-struggles-during-home',
 'https://www.asiaone.com/entertainment/worried-about-mental-wellness-suhaimi-yusof-shares-4-qs-help-him-stay-sane-and',
 'https://www.asiaone.com/lifestyle/singapore-mental-health-film-festival-2021-returns-may-22-panel-discussions-workshops',
 'https://www.asiaone.com/singapore/woman-19-rescued-after-sitting-27th-floor-ledge-sengkang-hdb-block-3-hours',
 'https://www.asiaone.com/entertainment/felicia-chin-felt-useless-couldnt-deal-stress-after-joining-mediacorp-and-developed',
 'https://www.asiaone.com/lifestyle/i-try-hypnotherapy-and-its-nothing-whats-shown-bling-empire',
 'https://www.asiaone.com/lifestyle/i-wanted-push-aircon-out-and-jump-down-it-entrepreneurs-epic-meltdown-changed-his-life',
 'https://www.asiaone.com/entertainment/che

In [14]:
len(urls)

38

In [35]:
articles = []

In [36]:

for j in range(20):
    data = get_page_data(urls[j],["singapore"])
    if(data != None):
        articles.append(data)
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [37]:
len(articles)

15

In [38]:

for k in range(20,len(urls)):
    data = get_page_data(urls[k],["singapore"])
    sleep(1)
    
    if(data != None):
        articles.append(data)
    print(k)

20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [39]:
len(articles)

24

In [40]:
df = pd.DataFrame(articles, columns=["title","url","author","date_str","content","is_Covid"])

In [41]:
df.index = df.index + 1

In [42]:
df

,title,url,author,date_str,content,is_Covid
1,'It took months to see the end of the tunnel':...,https://www.asiaone.com/lifestyle/it-took-mont...,Kimberly Anne Lim,"June 05, 2021published at 6:40 AM",When she first started gymnastics at the age o...,False
2,'Everyone watching makes me feel uneasy': Stud...,https://www.asiaone.com/singapore/everyone-wat...,Mardhiah Haslan,"June 01, 2021published at 12:00 AM","These days, Natasha Osman would sit in front o...",False
3,Singapore Mental Health Film Festival 2021 ret...,https://www.asiaone.com/lifestyle/singapore-me...,Alexander Kyle Thirumaran,"May 18, 2021published at 10:48 AM",The Covid-19 pandemic has heightened concerns ...,True
4,"Woman, 19, rescued after sitting on 27th-floor...",https://www.asiaone.com/singapore/woman-19-res...,Alexander Kyle Thirumaran,"April 09, 2021published at 7:15 AM",Residents in Sengkang craned their necks as th...,False
5,'I wanted to push the aircon out and jump down...,https://www.asiaone.com/lifestyle/i-wanted-pus...,Candice Cai,"January 09, 2021published at 3:20 AM",In his 30s and at the peak of his life and car...,False
6,Chew Chor Meng thought of ending his life whil...,https://www.asiaone.com/entertainment/chew-cho...,Bryan Lim,"November 13, 2020published at 6:15 AM","He once had it all — fame, money, success — bu...",False
7,Teetering on the edge: Woman rescued from ledg...,https://www.asiaone.com/singapore/teetering-ed...,Anna Tan,"October 29, 2020published at 7:13 AM",A 42-year-old woman sitting on a ledge at the ...,False
8,Eleanor Lee: Why is it an insult if I turn out...,https://www.asiaone.com/entertainment/eleanor-...,Kwok Kar Peng,"October 16, 2020published at 10:45 AM",When she was a little girl growing up in a sin...,False
9,Tragic coincidence: Same day as Korean actress...,https://www.asiaone.com/entertainment/tragic-c...,Bryan Lim,"September 16, 2020published at 8:08 AM","On the same day that Korean actress Oh In-hye,...",False
10,Woman allegedly threatens suicide after quarre...,https://www.asiaone.com/singapore/woman-allege...,Kimberly Anne Lim,"September 04, 2020published at 6:40 AM",A 50-year-old woman who allegedly threatened s...,False


In [43]:


path = 'AsiaOne.xlsx'

with pd.ExcelWriter(path) as writer:
    df.to_excel(writer, sheet_name='sheet1')
